In [1]:
from torch import nn
import torch
from torch.nn.functional import gumbel_softmax
import torch.nn.functional as F 

torch.manual_seed(0)

In [2]:
a = torch.tensor([[0, 0, 1, 0]])
b = torch.rand(4)
l = nn.Linear(4, 8)
l2 = nn.Linear(8, 4)

In [3]:
l.weight.T * a.T


tensor([[-0.0000, -0.0000, -0.0000,  0.0000,  0.0000, -0.0000, -0.0000, -0.0000],
        [ 0.0000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000, -0.0000, -0.0000],
        [-0.0099, -0.1511, -0.2061, -0.3390, -0.1029,  0.4527, -0.1949, -0.3493],
        [ 0.0000, -0.0000,  0.0000, -0.0000,  0.0000, -0.0000,  0.0000, -0.0000]],
       grad_fn=<MulBackward0>)

In [4]:
l2.weight.T * a

tensor([[-0.0000,  0.0000,  0.1600,  0.0000],
        [-0.0000,  0.0000,  0.1422,  0.0000],
        [ 0.0000,  0.0000, -0.2094, -0.0000],
        [ 0.0000, -0.0000,  0.1068, -0.0000],
        [-0.0000,  0.0000,  0.1941,  0.0000],
        [-0.0000, -0.0000, -0.0446,  0.0000],
        [ 0.0000, -0.0000,  0.0135,  0.0000],
        [ 0.0000, -0.0000,  0.0819,  0.0000]], grad_fn=<MulBackward0>)

In [5]:
b = torch.rand([64, 784])
res = F.gumbel_softmax(b, tau=1, hard=True, dim=-1)
res

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [6]:
import torch
import torch.nn.functional as F

def sample_gumbel(shape, eps=1e-20):
    U = torch.rand(shape).cuda()
    return -torch.tensor(torch.log(-torch.log(U + eps) + eps))

def gumbel_softmax_sample(logits, temperature):
    print(sample_gumbel(logits.size()))
    y = logits + sample_gumbel(logits.size())
    return F.softmax(y / temperature, dim=-1)

def gumbel_softmax(logits, topk, temperature):
    """
    input: [*, n_class]
    return: [*, n_class] an one-hot vector
    """
    y = gumbel_softmax_sample(logits, temperature)
    shape = y.size()
    ind = y.topk(topk).indices
    y_hard = torch.zeros_like(y).view(-1, shape[-1])
    y_hard.scatter_(1, ind.unsqueeze(0), 1)
    y_hard = y_hard.view(*shape)
    return (y_hard - y).detach() + y

# import math
# print(gumbel_softmax(Variable(torch.cuda.FloatTensor([[math.log(0.1), math.log(0.4), math.log(0.3), math.log(0.2)]] * 20000)),     0.8).sum(dim=0))

gumbel_softmax(torch.tensor([1, 0.13, 1, 0.03]).cuda(), 2, 1)

tensor([ 0.3709,  1.9028,  0.0851, -0.9270], device='cuda:0')


/tmp/ipykernel_295325/1988205471.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return -torch.tensor(torch.log(-torch.log(U + eps) + eps))


tensor([0., 1., 1., 0.], device='cuda:0')

In [7]:

def sample_gumbel(shape, eps=1e-20):
    U = torch.rand(shape).cuda()
    return -torch.log(-torch.log(U + eps) + eps)

def gumbel_softmax_sample(logits, temperature):
    y = logits + sample_gumbel(logits.size())
    return F.softmax(y / temperature, dim=-1)

def gumbel_softmax(logits, k, temperature):
    """
    ST-gumple-softmax
    input: [*, n_class]
    return: flatten --> [*, n_class] an one-hot vector
    """
    y = logits

    shape = y.size()
    ind = y.topk(k).indices
    print(ind)
    y_hard = torch.zeros_like(y).view(-1, shape[-1])
    y_hard.scatter_(1, ind.unsqueeze(0), 1)
    y_hard = y_hard.view(*shape)
    # Set gradients w.r.t. y_hard gradients w.r.t. y
    return (y_hard - y).detach() + y

gumbel_softmax(torch.tensor([1, 0.13, 1, 0.03, 0, 0]).cuda(), 2, 1)

tensor([2, 0], device='cuda:0')


tensor([1., 0., 1., 0., 0., 0.], device='cuda:0')

In [8]:
t = torch.rand((20))
print(t)

tensor([0.7196, 0.3458, 0.8343, 0.6468, 0.5813, 0.5788, 0.2981, 0.5921, 0.2240,
        0.3445, 0.8788, 0.4657, 0.8501, 0.3974, 0.7565, 0.4535, 0.7848, 0.6249,
        0.2660, 0.7630])


In [9]:


t = t.reshape((5, -1))
print(t)


# tm = F.gumbel_softmax(t, 1, True, dim=-1)
tm = torch.multinomial(t, 1)
print(tm)
tm.reshape((-1))


tensor([[0.7196, 0.3458, 0.8343, 0.6468],
        [0.5813, 0.5788, 0.2981, 0.5921],
        [0.2240, 0.3445, 0.8788, 0.4657],
        [0.8501, 0.3974, 0.7565, 0.4535],
        [0.7848, 0.6249, 0.2660, 0.7630]])
tensor([[1],
        [0],
        [2],
        [1],
        [0]])


tensor([1, 0, 2, 1, 0])

In [10]:
import torch

def deterministic_argmax(logits):
    return torch.argmax(logits).unsqueeze(0).float()

logits = torch.tensor([1.0, 2.0, 3.0])  # Example logits
result = torch.zeros_like(logits)
result[int(deterministic_argmax(logits).item())] = 1.0
print(result)

tensor([0., 0., 1.])


In [11]:
logits = torch.tensor([1.0, 2.0, 0.0, 1.0, 2.0, 3.0, 1.0, 2.0, 3.0, 2.0, 0.0, 1.0])
sparse = 3


In [12]:
logits = logits.reshape(sparse, -1)
logits

tensor([[1., 2., 0., 1.],
        [2., 3., 1., 2.],
        [3., 2., 0., 1.]])

In [13]:
ix = torch.argmax(logits, dim=-1)
ix

tensor([1, 1, 0])

In [14]:
torch.range(0, logits.numel()-1, sparse)

/tmp/ipykernel_295325/3240517964.py:1: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  torch.range(0, logits.numel()-1, sparse)


tensor([0., 3., 6., 9.])

In [15]:
ix = ix + torch.range(0, logits.numel()-1, logits.numel()/sparse)
ix.squeeze()

/tmp/ipykernel_295325/1556729594.py:1: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  ix = ix + torch.range(0, logits.numel()-1, logits.numel()/sparse)


tensor([1., 5., 8.])

In [16]:
logits.reshape(-1)[ix.squeeze().tolist()]

tensor([2., 3., 3.])

In [17]:
result = torch.zeros_like(logits)
result

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

In [18]:
logits = logits.reshape(-1)
logits

tensor([1., 2., 0., 1., 2., 3., 1., 2., 3., 2., 0., 1.])

In [19]:
a = torch.rand([16, 100])
a2 = torch.rand([100, 16])
b = torch.rand([1, 100])

(b @ a.T @ a2.T).shape

torch.Size([1, 100])

In [20]:
import torch
import torch.nn.functional as F

def create_self_attention_mask(seq_length):
    # Create a lower triangular matrix with ones below the diagonal
    mask = torch.tril(torch.ones(seq_length, seq_length)).unsqueeze(0).unsqueeze(1)
    return mask.bool()

# Example usage:
seq_length = 5
self_attention_mask = create_self_attention_mask(seq_length)
print("Self-attention mask:")
print(self_attention_mask)

Self-attention mask:
tensor([[[[ True, False, False, False, False],
          [ True,  True, False, False, False],
          [ True,  True,  True, False, False],
          [ True,  True,  True,  True, False],
          [ True,  True,  True,  True,  True]]]])


In [21]:
torch.tril(torch.ones(seq_length, seq_length)).bool()[:3, :3]

tensor([[ True, False, False],
        [ True,  True, False],
        [ True,  True,  True]])

In [22]:
def generate_square_subsequent_mask(sz):
    """Generate a mask to prevent attention to future positions."""
    mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

generate_square_subsequent_mask(3)

tensor([[0., -inf, -inf],
        [0., 0., -inf],
        [0., 0., 0.]])

In [23]:
a = torch.rand([5, 3])
b = torch.rand([7, 5])
a, b

(tensor([[0.9023, 0.4406, 0.4752],
         [0.1644, 0.1138, 0.7624],
         [0.6273, 0.3581, 0.0987],
         [0.2304, 0.8245, 0.7590],
         [0.2209, 0.0981, 0.5909]]),
 tensor([[0.1240, 0.4452, 0.6205, 0.0330, 0.7096],
         [0.8621, 0.0558, 0.3685, 0.2974, 0.6860],
         [0.6334, 0.3797, 0.0690, 0.5151, 0.5579],
         [0.4654, 0.4755, 0.6742, 0.2922, 0.7131],
         [0.6609, 0.0234, 0.6790, 0.1374, 0.6518],
         [0.2807, 0.3672, 0.1800, 0.6028, 0.1895],
         [0.3497, 0.2175, 0.7435, 0.7842, 0.6028]]))

In [24]:
s = torch.tensor([0, 1, 1, 1, 0]).unsqueeze(1)
print(s)
a[s]

tensor([[0],
        [1],
        [1],
        [1],
        [0]])


tensor([[[0.9023, 0.4406, 0.4752]],

        [[0.1644, 0.1138, 0.7624]],

        [[0.1644, 0.1138, 0.7624]],

        [[0.1644, 0.1138, 0.7624]],

        [[0.9023, 0.4406, 0.4752]]])

In [25]:
import torch

# Create a sample tensor
tensor = torch.tensor([[1, 2, 3],
                       [4, 5, 6],
                       [7, 8, 9]])

# Select one element from each row
selected_elements = tensor[torch.arange(tensor.size(0)), torch.tensor([2, 1, 2])]

print(selected_elements)

tensor([3, 5, 9])


In [26]:
torch.arange(tensor.size(0)), torch.tensor([2, 1, 2])

(tensor([0, 1, 2]), tensor([2, 1, 2]))

In [27]:
i = torch.rand([1024, 25])


In [28]:

ix = torch.argmax(i, dim=-1)
ix

tensor([10,  4,  8,  ..., 16,  8, 17])

In [29]:
zch = torch.zeros(i.shape)


In [30]:
ix.unsqueeze(0).shape

torch.Size([1, 1024])

In [31]:
zch.scatter(0, ix.unsqueeze(0), 1)
# zch.scatter(0, ix.unsqueeze(0), 1)

RuntimeError: Expected index [1, 1024] to be smaller than self [1024, 25] apart from dimension 0

In [ ]:
a = torch.tensor([[0,0,0,0], [0,0,0,0]])
b = torch.tensor([[1], [2]])

r = a.scatter(1, b, 1)
r

tensor([[0, 1, 0, 0],
        [0, 0, 1, 0]])

In [49]:
a = torch.arange(3)
torch.cat([a for _ in range(3)])

tensor([0, 1, 2, 0, 1, 2, 0, 1, 2])

In [71]:
a = torch.ones(2, 3,3)
b = torch.rand(2, 3,1)
a@b

tensor([[[1.1224],
         [1.1224],
         [1.1224]],

        [[0.8504],
         [0.8504],
         [0.8504]]])

In [72]:
b

tensor([[[0.3136],
         [0.3769],
         [0.4320]],

        [[0.1902],
         [0.0927],
         [0.5675]]])